# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
weather_df = pd.read_csv("../output_data/weather_csv.csv")
weather_df

,Lat,Long,City,Country,Date,Max Temp,Cloudiness,Humidity,Wind Speed
0,-3.6954,128.1814,Ambon City,ID,1666308834,82.54,20,78,3.44
1,22.1167,95.1333,Monywa,MM,1666308834,73.72,0,84,5.77
2,-54.8000,-68.3000,Ushuaia,AR,1666308834,44.22,20,61,25.32
3,42.4308,130.6434,Khasan,RU,1666308835,54.54,100,60,0.87
4,73.5069,80.5464,Dikson,RU,1666308835,32.11,100,98,16.08
...,...,...,...,...,...,...,...,...,...
577,-34.5833,150.8552,Flinders,AU,1666308821,71.19,100,92,5.01
578,2.0371,45.3438,Mogadishu,SO,1666309081,77.40,60,83,12.64
579,-0.1303,-67.0892,São Gabriel da Cachoeira,BR,1666309082,74.80,99,94,1.30
580,9.3500,42.8000,Jijiga,ET,1666309083,63.93,86,80,3.04


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)

In [4]:
locations = weather_df[["Lat", "Long"]]
humidity = weather_df["Humidity"].astype(float)

max_intensity = weather_df["Humidity"].max()
fig = gmaps.figure(center=(25,0), zoom_level=1.75)
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, max_intensity=max_intensity)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
#drill drill drill
#temp
perf_temp = weather_df[weather_df["Max Temp"] > 70]
perf_temp = perf_temp[perf_temp["Max Temp"] < 80]
#wind
perf_wind = perf_temp[perf_temp["Wind Speed"] < 10]
#clouds
perf_places = perf_wind[perf_wind["Cloudiness"] == 0]
perf_places

,Lat,Long,City,Country,Date,Max Temp,Cloudiness,Humidity,Wind Speed
1,22.1167,95.1333,Monywa,MM,1666308834,73.72,0,84,5.77
28,13.1667,44.5833,Muhafazat Lahij,YE,1666308846,75.31,0,72,4.54
84,-23.8650,35.3833,Inhambane,MZ,1666308871,70.84,0,84,5.30
88,42.2068,-121.7372,Altamont,US,1666308873,78.78,0,19,6.91
152,43.1330,-115.6912,Mountain Home,US,1666308599,72.88,0,17,3.44
166,37.6775,-113.0619,Cedar City,US,1666308844,70.77,0,16,3.44
234,26.3260,43.9750,Buraidah,SA,1666308783,70.25,0,14,7.14
274,40.9730,-117.7357,Winnemucca,US,1666308949,78.85,0,8,5.75
282,24.4810,68.9859,Kadhan,PK,1666308952,75.06,0,63,5.37
286,36.1156,-97.0584,Stillwater,US,1666308953,79.07,0,22,3.44


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
hotel_df = perf_places.copy()
hotel_df = hotel_df.reset_index()
del hotel_df["index"]

hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,Lat,Long,City,Country,Date,Max Temp,Cloudiness,Humidity,Wind Speed,Hotel Name
0,22.1167,95.1333,Monywa,MM,1666308834,73.72,0,84,5.77,
1,13.1667,44.5833,Muhafazat Lahij,YE,1666308846,75.31,0,72,4.54,
2,-23.8650,35.3833,Inhambane,MZ,1666308871,70.84,0,84,5.30,
3,42.2068,-121.7372,Altamont,US,1666308873,78.78,0,19,6.91,
4,43.1330,-115.6912,Mountain Home,US,1666308599,72.88,0,17,3.44,
5,37.6775,-113.0619,Cedar City,US,1666308844,70.77,0,16,3.44,
6,26.3260,43.9750,Buraidah,SA,1666308783,70.25,0,14,7.14,
7,40.9730,-117.7357,Winnemucca,US,1666308949,78.85,0,8,5.75,
8,24.4810,68.9859,Kadhan,PK,1666308952,75.06,0,63,5.37,
9,36.1156,-97.0584,Stillwater,US,1666308953,79.07,0,22,3.44,


In [7]:
#oh look another loop
lat = hotel_df["Lat"]
long = hotel_df["Long"]
range_x = 10

for x in range(range_x):
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    params = {
        "location": f"{lat[x]},{long[x]}", 
        "radius": 5000,
        "type": "lodging",
        "keyword": "Hotel",
        "key": g_key,
    }
    response = requests.get(base_url, params=params).json()
    
    try:
        results = response["results"][0]["name"]
        print(f"The closest hotel is {results}.")
        hotel_df.loc[hotel_df["City"] == hotel_df["City"][x], "Hotel Name"] = results        
    except (IndexError):
        print("No hotels within 5000 meters, skipping.")
    print("----")

The closest hotel is Shwe Taung Tan Lake View Hotel.
----
No hotels within 5000 meters, skipping.
----
The closest hotel is Hotel Casa Do Capitão.
----
The closest hotel is Fairfield Inn & Suites by Marriott Klamath Falls.
----
The closest hotel is Mountain Home Inn.
----
The closest hotel is Courtyard by Marriott Cedar City.
----
The closest hotel is Mövenpick Qassim.
----
The closest hotel is Fairfield Inn & Suites by Marriott Winnemucca.
----
No hotels within 5000 meters, skipping.
----
The closest hotel is Home2 Suites by Hilton Stillwater.
----


In [8]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Long"]]

In [9]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))